In [1]:
import optuna
from transformers import(
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BertTokenizer, 
    BertForMaskedLM

)
from datasets import DatasetDict, Dataset
import pandas as pd
import evaluate
import numpy as np
from huggingface_hub import notebook_login
# from optuna.integration import PyTorchLightningPruningCallback
import logging
import sys
import torch
import os
from sklearn.model_selection import train_test_split

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"using device: {device}")
# notebook_login()

In [2]:
df = pd.read_pickle("data/clean_food.pkl")

recipes = list(df['ingredients'].apply(lambda x: ', '.join(x)))
dataset = Dataset.from_dict({'text': recipes})
dataset


Dataset({
    features: ['text'],
    num_rows: 2908435
})

In [3]:
# 1. Load a pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'  # Or any other BERT variant
tokenizer = BertTokenizer.from_pretrained(model_name)



In [4]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, return_tensors = 'pt')

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format("torch")


Map:   0%|          | 0/2908435 [00:00<?, ? examples/s]

In [5]:
input_ids = tokenized_dataset['input_ids']

In [6]:
attention_mask = tokenized_dataset['attention_mask']

In [7]:
token_dataset = Dataset.from_dict({
    "input_ids": input_ids,
    "attention_mask": attention_mask
})

KeyboardInterrupt: 

In [ ]:
# dataset = Dataset.from_dict({"input_ids": tokenized_dataset["input_ids"],
#                              "attention_mask": tokenized_dataset['attention_mask']})

In [ ]:
def mask_tokens(examples):
    input_ids = examples["input_ids"]
    batch_size, seq_length = input_ids.size()
    labels = input_ids.clone()
    # We sample a few tokens in each sequence for MLM training (15% by default)
    probability_matrix = torch.full((batch_size, seq_length), 0.15)
    special_tokens_mask = (input_ids == tokenizer.cls_token_id) | (input_ids == tokenizer.sep_token_id) | (input_ids == tokenizer.pad_token_id)

    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens
    input_ids[masked_indices] = tokenizer.mask_token_id

    return {"input_ids": input_ids, "attention_mask": examples["attention_mask"], "labels": labels}

In [ ]:
# sample_size = 1000
# masked_dataset = dataset.map(mask_tokens, batched=True)
# masked_dataset.set_format("torch")